In [3]:
import pandas as pd

In [25]:
pd.read_csv("enhance_ocod/data/training_data/ground_truth_dev_set_labels.csv")

,label,start,end,text,input:text,input:datapoint_id
0,street_number,0,3,207,"207 sloane street, london (sw1x 9qx)",53574
1,street_name,4,17,sloane street,"207 sloane street, london (sw1x 9qx)",53574
2,city,19,25,london,"207 sloane street, london (sw1x 9qx)",53574
3,postcode,27,35,sw1x 9qx,"207 sloane street, london (sw1x 9qx)",53574
4,unit_type,0,9,apartment,"apartment 533, block 11 spectrum, blackfriars ...",17501
...,...,...,...,...,...,...
73294,street_number,8,10,49,"flat a, 49 norcott road, stoke newington, (n16...",76011
73295,street_name,11,23,norcott road,"flat a, 49 norcott road, stoke newington, (n16...",76011
73296,street_number,25,30,stoke,"flat a, 49 norcott road, stoke newington, (n16...",76011
73297,street_number,31,40,newington,"flat a, 49 norcott road, stoke newington, (n16...",76011


In [19]:
from typing import List, Dict, Any, Iterator
from collections import defaultdict

def convert_df_to_gliner_format(df: pd.DataFrame) -> List[Dict]:
    """
    Convert DataFrame data in the given format to GLiNER training format.
    
    Args:
        df: DataFrame containing the data
        
    Returns:
        List of dictionaries in GLiNER format
    """
    
    # Group by the input text and datapoint_id, but deduplicate spans
    grouped_data = {}
    for _, row in df.iterrows():
        key = (row['input:text'], row['input:datapoint_id'])
        
        if key not in grouped_data:
            grouped_data[key] = {
                'text': row['input:text'],
                'spans': set()  # Using a set to prevent duplicates
            }
        
        # Convert span to a hashable tuple to use in a set
        span_tuple = (row['text'], row['start'], row['end'], row['label'])
        grouped_data[key]['spans'].add(span_tuple)
    
    # Convert back to the GLiNER format
    gliner_data = []
    for data in grouped_data.values():
        # Convert set of tuples back to list of dictionaries
        spans = [
            {'text': text, 'start': start, 'end': end, 'label': label}
            for text, start, end, label in data['spans']
        ]
        
        gliner_data.append({
            'text': data['text'],
            'spans': spans
        })
    
    return gliner_data

In [26]:
#test = pd.read_csv("enhance_ocod/data/training_data/parsed_ground_truth_complete.csv")
data = pd.read_csv("data/training_data/ground_truth_dev_set_labels.csv")
data = pd.read_csv("data/training_data/ground_truth_test_set_labels.csv")

In [28]:
out_data = convert_df_to_gliner_format(data)

In [29]:
len(out_data)

1000

In [30]:
import json
with open('data/training_data_test.jsonl', 'w', encoding='utf-8') as f:
    json.dump(out_data, f, ensure_ascii=False, indent=2)

Parsing: Flat 3, 25 Test Road, London
- UNIT (flat): 'Flat 3'
- UNIT (number): '25'
- STREET (road_street): 'Road'
